## Optimización combinatoria

Un vendedor debe hacer un recorrido por todas y cada una de las capitales de los 32 estados de los Estados Unidos Mexicanos.

Utilice colonias de hormigas y algoritmos genéticos para encontrar el orden óptimo. El costo de desplazamiento entre ciudades es la suma del valor de la hora del vendedor (es un parámetro que debe estudiarse), el costo de los peajes y el costo del combustible. Cada equipo debe definir en qué carro hace el recorrido el vendedor y de allí extraer el costo del combustible.

Adicionalmente represente con un gif animado o un video cómo se comporta la mejor solución usando un gráfico del recorrido en el mapa de México.

In [3]:
!pip install -r requirements.txt



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Julia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import folium
import numpy as np
from IPython.display import display
import random
from folium import plugins
import seaborn as sns
import matplotlib.pyplot as plt



## Carga y preparación de datos necesarios

El desarrollo del trabajo cuentan con 5 archivos, que servirán para la optimización del recorrido y la actividad. 

coordenadas.csv: Contiene las coordenadas de las capitales de los estados de México.

tiempos.csv: Contiene los tiempos de desplazamiento entre las capitales de los estados de México

distancias.csv: Contiene las distancias entre las capitales de los estados de México.

peajes.csv: Contiene los peajes entre las capitales de los estados de México.

Fuente coordenadas: INEGI - Marco Geoestadístico Nacional 2020
https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=889463807469


In [5]:
tiempos = pd.read_csv('./datos/input/tiempos_conduccion.csv',index_col=0)
distancias = pd.read_csv('./datos/input/distancias.csv',index_col=0)
coordenadas = pd.read_csv('./datos/input/coordenadas.csv',header=None)
coordenadas = coordenadas.rename(columns={0:'latitude', 1:'longitude'})
nombres_estados = list(tiempos.columns)
peajes = pd.read_csv('./datos/input/peajes.csv',index_col=0)

In [6]:
def tiempo_dec(tiempo_str):
    if type(tiempo_str) == str:
        hh,mm = map(int, tiempo_str.split(':'))
        return round(hh+mm/60.0,4)
    else:
        return 0
tiempos = tiempos.fillna(0)
tiempos
tiempos_ = tiempos.copy().map(tiempo_dec)
times = tiempos_.to_numpy()

Visualización del mapa de México usando la librería folium, ubicando cada una de sus capitales.

In [7]:
import folium
# Crear mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

# Agregar marcadores para cada capital
for capital, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude']):
    folium.Marker(
        location=[lat, lon],
        popup=capital,
        tooltip=capital
    ).add_to(m)

# Mostrar el mapa
m

## Cálculo de salario conductor

Se asume que los conductores ganan un salario basado en horas trabajadas, una vez con los datos de tiempo de desplazamiento fue necesario consultar el salario promedio de los conductores de carga para calcular su salario a través del recorrido que se creará.

Basado en la información público del CONASAMI mexicano, se hallo que el salario de un chofer repartidor de un vehículo con capacidad como el Renault Kangoo es en promedio 46 MXN. 

Esta información se puede visualizar en https://www.gob.mx/cms/uploads/attachment/file/873886/Tabla_de_Salarios_M_nimos_2024.pdf?trk=article-ssr-frontend-pulse_little-text-block.

In [8]:
val_por_hora = times*46
print(val_por_hora)

#arreglo de numpy val_hora_tiempo a csv
np.savetxt('./datos/output/val_por_hora.csv', val_por_hora, delimiter=',', fmt='%.2f')

[[   0.     1357.7682 1346.2682 ...  568.1    1372.3318   94.3   ]
 [1330.9318    0.     1190.6318 ... 2072.3    2875.7682 1251.9682]
 [1382.3    1180.6682    0.     ... 1832.3318 2636.5682 1171.4682]
 ...
 [ 525.1682 2083.8    1849.2    ...    0.      920.7682  619.4682]
 [1314.8318 2873.4682 2638.8682 ...  910.0318    0.     1409.1318]
 [  89.7    1272.6682 1138.5    ...  657.0318 1460.5       0.    ]]


## Peajes
El costo de los peajes de un estado a otro puede encontrarse en la página [tarifascapufe.com.mx.](https://tarifascapufe.com.mx/traza-tu-ruta/) y se recopiló la información con uso de IA para convertirlo en el archivo csv de "peajes.csv"

In [9]:
peajes = peajes.fillna(0)
peajes_np = peajes.to_numpy()
peajes_np

array([[   0., 1588., 1629., ..., 1597., 2737.,   44.],
       [1588.,    0.,    0., ..., 5078., 6218., 1486.],
       [1629.,    0.,    0., ..., 4143., 5283., 1527.],
       ...,
       [1597., 5078., 4143., ...,    0., 1232.,  947.],
       [2737., 6218., 5283., ..., 1232.,    0., 2171.],
       [  44., 1486., 1527., ...,  947., 2171.,    0.]])

### Costo del combustible

El precio de la gasolina regular en México para 2024 varía dependiendo de la región y otros factores. En promedio, el precio por litro de gasolina regular es de aproximadamente 23.66 pesos. Sin embargo, este precio puede fluctuar ligeramente según la ubicación y las condiciones del mercado. https://www.renault.com.mx/vehiculos-utilitarios/kangoo/datos-tecnicos.html

Elegimos el Renault Kangoo por sus características para carga y distribución para paquetes (Es uno de los carros más usados para transportes de envios nacionales e internacioles)

- Contiene un tanque con capacidad de 50 litros.
- Su rendimiento de cosbustible es de 16 km/1 


Para calcular el costo de combustible se usa la Fórmula de costo de combustible para un viaje por carretera.


costo = (distancia / rendimiento) * precio_gasolina

Donde:

Distancia de un estado a otro se refiere a la distancia total que se recorrerá.
Eficiencia del combustible es el rendimiento del vehículo en términos de kilómetros por litro (km/l) o millas por galón (mpg).
Precio de la gasolina es el costo por unidad de combustible (por litro o por galón).


In [10]:
distancias = distancias.fillna(0)
distancias_np = distancias.to_numpy()

costo_combustible = (distancias_np/16.1)*23.66 
costo_combustible

array([[   0.        , 3161.03478261, 1967.74782609, ..., 1093.35652174,
        2620.23478261,  173.40869565],
       [3155.15652174,    0.        , 1989.79130435, ..., 4188.26086957,
        5715.13913043, 2986.15652174],
       [1967.74782609, 1989.79130435,    0.        , ..., 2818.62608696,
        4345.50434783, 1739.96521739],
       ...,
       [1087.47826087, 4185.32173913, 2815.68695652, ...,    0.        ,
        1641.50434783, 1262.35652174],
       [2630.52173913, 5726.89565217, 4358.73043478, ..., 1640.03478261,
           0.        , 2803.93043478],
       [ 171.93913043, 2996.44347826, 1745.84347826, ..., 1265.29565217,
        2790.70434783,    0.        ]])

## Cálculo de costo total
Para calcular el costo total de ir de un estado a otro, se consideran tres componentes principales:

1. Costo del tiempo del conductor: 
   - Salario base del conductor: 46 MXN/hora
   - Multiplicado por el tiempo de viaje entre estados

2. Costo del combustible:
   - Precio de gasolina: 23.66 MXN/L
   - Rendimiento del vehículo (Renault Kangoo): 16 km/L
   - Calculado como: (distancia / rendimiento) * precio_gasolina

3. Costo de peajes:
   - Obtenidos directamente de la matriz de peajes

La fórmula final para el costo entre estados es:

$Costo_{total} = (Salario_{base} * Tiempo_{viaje}) + (\frac{Distancia}{Rendimiento} * Precio_{gasolina}) + Peaje$

Donde:
- $Salario_{base} = 46$ MXN/hora
- $Rendimiento = 16$ km/L
- $Precio_{gasolina} = 23.66$ MXN/L


In [11]:
costo_total = val_por_hora + peajes + costo_combustible
print(costo_total)

np.savetxt('./datos/output/costo_total.csv', costo_total, delimiter=',', fmt='%.2f')

                            Aguascalientes      Mexicali        La Paz  \
Costo de peajes                                                          
Aguascalientes                    0.000000   6106.802983   4943.016026   
Mexicali                       6074.088322      0.000000   3180.423104   
La Paz                         4979.047826   3170.459504      0.000000   
San Francisco de Campeche      6281.727452  10495.869130  11878.204348   
Tuxtla Gutiérrez               5335.816026  13471.523104  10932.288322   
Chihuahua                      3066.073913   2998.062235   2782.940496   
CDMX                           2064.701365  10213.288322   7675.523104   
Saltillo                       1138.529070   5529.223104   5721.614409   
Colima                         2156.091304   8652.063852   6114.298635   
Victoria de Durango            1101.136148   6063.518757   3668.283974   
Guanajuato                      714.581243   8901.201365   6363.436148   
Chilpancingo de los Bravo      3387.13

# Optimización con algoritmo de colonia de hormigas

In [12]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)

# Parámetros del algoritmo de colonia de hormigas
n_states = 32  # Número de estados mexicanos
alpha = 1      # Influencia de las feromonas en la decisión
beta = 2       # Influencia de la visibilidad (costos de viaje)
evaporation_rate = 0.5  # Tasa de evaporación de feromonas
pheromone_constant = 100  # Constante de depósito de feromonas
n_ants = n_states  # Una hormiga por cada estado

# Convertir DataFrame a array numpy si es necesario
if isinstance(costo_total, pd.DataFrame):
    costo_total = costo_total.to_numpy()

# Inicialización de matrices de feromonas y visibilidad
pheromones = np.ones((n_states, n_states))
visibility = np.zeros((n_states, n_states))

# Calcular matriz de visibilidad (inversa del costo)
np.fill_diagonal(visibility, 0)  
for i in range(n_states):
    for j in range(n_states):
        if i != j and costo_total[i,j] != 0:
            visibility[i,j] = 1 / costo_total[i,j]

# Variables para almacenar la mejor solución
mejor_ruta = None
mejor_costo = float('inf')

# Iteraciones principales del algoritmo
for iteracion in range(100):  
    todas_rutas = []
    todos_costos = []

    # Ciclo para cada hormiga
    for hormiga in range(n_ants):
        # Selección aleatoria del estado inicial
        estado_actual = random.randint(0, n_states - 1)
        estados_visitados = [estado_actual]
        costo_ruta = 0  # Cambiado de 'costo' a 'costo_ruta'

        # Construcción de la ruta
        for paso in range(n_states - 1):
            probabilidades = []
            for siguiente_estado in range(n_states):
                if siguiente_estado not in estados_visitados:
                    prob = (pheromones[estado_actual, siguiente_estado] ** alpha) * \
                          (visibility[estado_actual, siguiente_estado] ** beta)
                    probabilidades.append((siguiente_estado, prob))

            if probabilidades:  
                prob_total = sum(prob for _, prob in probabilidades)
                if prob_total > 0:  
                    probabilidades = [(estado, prob/prob_total) for estado, prob in probabilidades]
                    siguiente_estado = random.choices(
                        [estado for estado, _ in probabilidades],
                        weights=[prob for _, prob in probabilidades]
                    )[0]
                else:
                    siguiente_estado = random.choice([estado for estado, _ in probabilidades])
            else:
                break

            # Actualización de costos y estado
            costo_ruta += costo_total[estado_actual, siguiente_estado]
            estado_actual = siguiente_estado
            estados_visitados.append(estado_actual)

        # Completar el ciclo volviendo al inicio
        if len(estados_visitados) == n_states:  
            costo_ruta += costo_total[estado_actual, estados_visitados[0]]
            todas_rutas.append(estados_visitados)
            todos_costos.append(costo_ruta)

            # Actualizar mejor ruta si corresponde
            if costo_ruta < mejor_costo:
                mejor_costo = costo_ruta
                mejor_ruta = estados_visitados.copy()

    # Actualización de feromonas
    pheromones *= (1 - evaporation_rate)  
    for ruta, costo in zip(todas_rutas, todos_costos):
        for i in range(len(ruta) - 1):
            pheromones[ruta[i], ruta[i+1]] += pheromone_constant / costo
        pheromones[ruta[-1], ruta[0]] += pheromone_constant / costo

# Presentación de resultados
if mejor_ruta is None:
    print("No se encontró una solución factible")
else:
    nombres_ruta = [nombres_estados[i] for i in mejor_ruta]
    nombres_ruta.append(nombres_ruta[0])  

    print("\nMejor ruta encontrada (incluyendo retorno al origen):")
    print("→ ".join(nombres_ruta))

    print("\nDesglose de costos por tramo:")
    costo_total_ruta = 0  # Cambiado de 'costo_total' a 'costo_total_ruta'
    print("\n{:<35} {:<35} {:<15}".format("Estado Origen", "Estado Destino", "Costo (MXN)"))
    print("-" * 85)

    for i in range(len(mejor_ruta)):
        estado_actual = mejor_ruta[i]
        siguiente_estado = mejor_ruta[0] if i == len(mejor_ruta) - 1 else mejor_ruta[i + 1]
        costo_tramo = costo_total[estado_actual, siguiente_estado]
        costo_total_ruta += costo_tramo
        print("{:<35} {:<35} ${:<15,.2f}".format(
            nombres_estados[estado_actual],
            nombres_estados[siguiente_estado],
            costo_tramo
        ))

    print(f"\nTotal de estados visitados: {len(nombres_ruta)}")
    print(f"Costo total del recorrido: ${costo_total_ruta:,.2f} MXN")


Mejor ruta encontrada (incluyendo retorno al origen):
Tuxtla Gutiérrez→ Oaxaca de Juárez→ Heroica Puebla de Zaragoza→ Tlaxcala de Xicohténcatl→ Pachuca de Soto→ CDMX→ Cuernavaca→ Chilpancingo de los Bravo→ Toluca de Lerdo→ Santiago de Querétaro→ Guanajuato→ San Luis Potosí→ Ciudad Victoria→ Monterrey→ Saltillo→ Zacatecas→ Aguascalientes→ Victoria de Durango→ Chihuahua→ Hermosillo→ Mexicali→ La Paz→ Culiacán Rosales→ Tepic→ Guadalajara→ Colima→ Morelia→ Xalapa-Enríquez→ Villahermosa→ San Francisco de Campeche→ Mérida→ Chetumal→ Tuxtla Gutiérrez

Desglose de costos por tramo:

Estado Origen                       Estado Destino                      Costo (MXN)    
-------------------------------------------------------------------------------------
Tuxtla Gutiérrez                    Oaxaca de Juárez                    $1,400.39       
Oaxaca de Juárez                    Heroica Puebla de Zaragoza          $1,104.87       
Heroica Puebla de Zaragoza          Tlaxcala de Xicohténcatl     

In [13]:
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}

In [14]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}


# Ruta del recorrido en orden
ruta = nombres_ruta  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="red", weight=4, opacity=0.8).add_to(m)
m

## Optimización usando algoritmos genéticos

In [15]:
import numpy as np
import random

random.seed(42)
np.random.seed(42)

# Parámetros del problema
n_states = 32  # Número de estados
n_population = 100  # Tamaño de la población
n_generations = 200  # Número de generaciones
mutation_rate = 0.01  # Tasa de mutación

# Genera una ruta aleatoria
def generate_route():
    route = list(range(n_states))
    random.shuffle(route)
    return route

# Calcula el costo total de una ruta
def calculate_cost(route):
    total_cost = 0
    segments = []
    
    # Calcula el costo entre cada par de estados consecutivos
    for i in range(n_states - 1):
        cost = costo_total[route[i], route[i + 1]]
        total_cost += cost
        segments.append((route[i], route[i + 1], cost))
    
    # Añade el costo del retorno al estado inicial
    cost_return = costo_total[route[-1], route[0]]
    total_cost += cost_return
    segments.append((route[-1], route[0], cost_return))
    
    return total_cost, segments

# Genera la población inicial
population = [generate_route() for _ in range(n_population)]

# Seguimiento de la mejor ruta en cada generación
best_costs_per_generation = []

# Algoritmo evolutivo
for generation in range(n_generations):
    # Evaluación de la aptitud
    fitness_scores = [(route, *calculate_cost(route)) for route in population]
    fitness_scores.sort(key=lambda x: x[1])  # Ordena por costo
    
    # Guarda el mejor costo de esta generación
    best_costs_per_generation.append(fitness_scores[0][1])

    # Selección (elitismo: conserva las mejores rutas)
    elite_size = int(0.1 * n_population)  # Mantener el 10% de la mejor población
    new_population = [route for route, _, _ in fitness_scores[:elite_size]]
    
    # Generación de nuevos individuos (crossover y mutación)
    while len(new_population) < n_population:
        # Selección de padres
        parent1, parent2 = random.choices(population, k=2)
        
        # Crossover (Orden basado en cruce)
        crossover_point = random.randint(1, n_states - 2)
        child = parent1[:crossover_point] + [state for state in parent2 if state not in parent1[:crossover_point]]
        
        # Mutación
        if random.random() < mutation_rate:
            i, j = random.sample(range(n_states), 2)
            child[i], child[j] = child[j], child[i]  # Intercambia dos estados aleatorios
        
        new_population.append(child)
    
    population = new_population  # Actualiza la población para la siguiente generación

# Mejor ruta encontrada
best_route, best_cost, best_segments = min(fitness_scores, key=lambda x: x[1])
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

# Mostrar resultados
print("\nMejor ruta encontrada (incluyendo retorno al inicio):")
print("→ ".join(best_route_names))

print("\nDesglose del costo por segmento:")
print("\n{:<35} {:<35} {:<15}".format("Origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for origin, destination, cost in best_segments:
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[origin],
        nombres_estados[destination],
        cost
    ))

print("\nEstadísticas del viaje:")
print(f"Número total de estados visitados: {len(best_route_names)} (incluyendo el retorno al inicio)")
print(f"Costo total del viaje: ${best_cost:,.2f} MXN")
print(f"Costo promedio por segmento: ${best_cost/len(best_route):,.2f} MXN")

# Estadísticas de la evolución
print("\nEstadísticas de la evolución:")
print(f"Costo inicial (mejor de la primera generación): ${best_costs_per_generation[0]:,.2f} MXN")
print(f"Costo final (mejor de la última generación): ${best_costs_per_generation[-1]:,.2f} MXN")
print(f"Mejora total: ${(best_costs_per_generation[0] - best_costs_per_generation[-1]):,.2f} MXN")
print(f"Porcentaje de mejora: {((best_costs_per_generation[0] - best_costs_per_generation[-1]) / best_costs_per_generation[0] * 100):,.2f}%")


Mejor ruta encontrada (incluyendo retorno al inicio):
Culiacán Rosales→ Morelia→ Tlaxcala de Xicohténcatl→ Tuxtla Gutiérrez→ Oaxaca de Juárez→ Chilpancingo de los Bravo→ Heroica Puebla de Zaragoza→ Pachuca de Soto→ Zacatecas→ Victoria de Durango→ Chihuahua→ Saltillo→ San Luis Potosí→ Santiago de Querétaro→ Tepic→ Guadalajara→ Aguascalientes→ San Francisco de Campeche→ Mérida→ Chetumal→ Villahermosa→ Xalapa-Enríquez→ Ciudad Victoria→ Toluca de Lerdo→ CDMX→ Cuernavaca→ Guanajuato→ Colima→ Monterrey→ Mexicali→ Hermosillo→ La Paz→ Culiacán Rosales

Desglose del costo por segmento:

Origen                              Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Culiacán Rosales                    Morelia                             $4,869.45       
Morelia                             Tlaxcala de Xicohténcatl            $1,908.07       
Tlaxcala de Xicohténcatl            Tuxtla Gutiérrez          

In [16]:
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}

In [17]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}
# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="red", weight=4, opacity=0.8).add_to(m)
m

Al comparar ambos algoritmos, se observa que el algoritmo de colonia de hormigas ofrece una optimización más eficiente de los costos de transporte en la ruta óptima en comparación con los algoritmos genéticos. Esto se refleja en el costo final de cada ruta: el algoritmo de colonia de hormigas logró un costo de \$39,056.11 MXN, mientras que los algoritmos genéticos presentaron un costo total de \$64,287.88 MXN.

# Partiendo desde el mismo punto de partida

## Algoritmos genéticos

In [18]:
import numpy as np
import random

random.seed(42)
np.random.seed(42)

# Parámetros del problema
n_states = 32  # Número de estados
n_population = 100  # Tamaño de la población
n_generations = 200  # Número de generaciones
mutation_rate = 0.01  # Tasa de mutación

# Genera una ruta aleatoria comenzando desde Ciudad de México (índice 0)
def generate_route():
    route = list(range(1, n_states))  # Empieza con los demás estados (excepto Ciudad de México)
    random.shuffle(route)
    route = [0] + route  # Inserta Ciudad de México (índice 0) al inicio
    return route

# Calcula el costo total de una ruta
def calculate_cost(route):
    total_cost = 0
    segments = []

    # Calcula el costo entre cada par de estados consecutivos
    for i in range(n_states - 1):
        cost = costo_total[route[i], route[i + 1]]
        total_cost += cost
        segments.append((route[i], route[i + 1], cost))

    # Añade el costo del retorno al estado inicial
    cost_return = costo_total[route[-1], route[0]]
    total_cost += cost_return
    segments.append((route[-1], route[0], cost_return))

    return total_cost, segments

# Genera la población inicial
population = [generate_route() for _ in range(n_population)]

# Seguimiento de la mejor ruta en cada generación
best_costs_per_generation = []

# Algoritmo evolutivo
for generation in range(n_generations):
    # Evaluación de la aptitud
    fitness_scores = [(route, *calculate_cost(route)) for route in population]
    fitness_scores.sort(key=lambda x: x[1])  # Ordena por costo

    # Guarda el mejor costo de esta generación
    best_costs_per_generation.append(fitness_scores[0][1])

    # Selección (elitismo: conserva las mejores rutas)
    elite_size = int(0.1 * n_population)  # Mantener el 10% de la mejor población
    new_population = [route for route, _, _ in fitness_scores[:elite_size]]

    # Generación de nuevos individuos (crossover y mutación)
    while len(new_population) < n_population:
        # Selección de padres
        parent1, parent2 = random.choices(population, k=2)

        # Crossover (Orden basado en cruce)
        crossover_point = random.randint(1, n_states - 2)
        child = parent1[:crossover_point] + [state for state in parent2 if state not in parent1[:crossover_point]]

        # Mutación
        if random.random() < mutation_rate:
            i, j = random.sample(range(n_states), 2)
            child[i], child[j] = child[j], child[i]  # Intercambia dos estados aleatorios

        new_population.append(child)

    population = new_population  # Actualiza la población para la siguiente generación

# Mejor ruta encontrada
best_route, best_cost, best_segments = min(fitness_scores, key=lambda x: x[1])
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

print("\nRuta óptima encontrada (incluyendo regreso al punto de partida):")
print(" → ".join(best_route_names))

print("\nDetalle de costos por tramo:")
print("\n{:<35} {:<35} {:<15}".format("Punto de origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for origin, destination, cost in best_segments:
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[origin],
        nombres_estados[destination],
        cost
    ))

print("\nResumen del recorrido:")
print(f"Total de estados visitados: {len(best_route_names)} (incluyendo el retorno al inicio)")
print(f"Costo total del recorrido: ${best_cost:,.2f} MXN")
print(f"Costo medio por trayecto: ${best_cost/len(best_route):,.2f} MXN")

# Estadísticas de la evolución
print("\nAnálisis del proceso evolutivo:")
print(f"Costo inicial (mejor ruta en la primera generación): ${best_costs_per_generation[0]:,.2f} MXN")
print(f"Costo final (mejor ruta en la última generación): ${best_costs_per_generation[-1]:,.2f} MXN")
print(f"Reducción de costos total: ${(best_costs_per_generation[0] - best_costs_per_generation[-1]):,.2f} MXN")
print(f"Porcentaje de mejora en la optimización: {((best_costs_per_generation[0] - best_costs_per_generation[-1]) / best_costs_per_generation[0] * 100):,.2f}%")



Ruta óptima encontrada (incluyendo regreso al punto de partida):
Guanajuato → Santiago de Querétaro → Saltillo → Aguascalientes → Zacatecas → Guadalajara → Colima → Morelia → Chilpancingo de los Bravo → Heroica Puebla de Zaragoza → Oaxaca de Juárez → Tuxtla Gutiérrez → Chetumal → Mérida → San Francisco de Campeche → Xalapa-Enríquez → Tlaxcala de Xicohténcatl → Toluca de Lerdo → Victoria de Durango → Culiacán Rosales → La Paz → Chihuahua → San Luis Potosí → Ciudad Victoria → Villahermosa → Pachuca de Soto → Cuernavaca → CDMX → Monterrey → Mexicali → Hermosillo → Tepic → Guanajuato

Detalle de costos por tramo:

Punto de origen                     Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Guanajuato                          Santiago de Querétaro               $568.60         
Santiago de Querétaro               Saltillo                            $1,614.01       
Saltillo                     

In [19]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}


# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="red", weight=4, opacity=0.8).add_to(m)
m.save("./Trayectorias_fijas_simuladas_para_algoritmos_genéticos.html")
display(m)

## Colonia de hormigas

In [20]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)

# Parámetros del algoritmo
n_states = 32  # Número de estados
alpha = 1      # Influencia de las feromonas
beta = 2       # Influencia de la visibilidad (tiempos de viaje)
evaporation_rate = 0.5
pheromone_constant = 100
n_ants = n_states  # Usar una hormiga por estado

# Inicializa la matriz de feromonas
pheromones = np.ones((n_states, n_states))  # Cantidad inicial de feromonas

# Cálculo de visibilidad (inversa del tiempo de viaje, para que tiempos menores sean más atractivos)
visibility = 1 / costo_total
visibility[visibility == np.inf] = 0  # Evitar divisiones por cero

# Ejecución del algoritmo
best_route = None
best_cost = float('inf')

for iteration in range(100):  # Número de iteraciones para el algoritmo
    all_routes = []
    all_costs = []

    # Para cada hormiga
    for ant in range(n_ants):
        # Fijar el estado inicial a Ciudad de México (índice 0)
        current_state = 0  # Ciudad de México
        visited_states = [current_state]
        cost = 0

        # Construcción de la ruta completa
        for step in range(n_states - 1):
            # Calcular probabilidades para el próximo estado
            probabilities = []
            for next_state in range(n_states):
                if next_state not in visited_states:
                    prob = (pheromones[current_state][next_state] ** alpha) * (visibility[current_state][next_state] ** beta)
                    probabilities.append((next_state, prob))

            # Seleccionar el próximo estado basado en las probabilidades
            next_state = random.choices(
                [state for state, _ in probabilities],
                weights=[prob for _, prob in probabilities]
            )[0]

            # Actualizar el costo y el estado actual
            cost += costo_total[current_state][next_state]
            current_state = next_state
            visited_states.append(current_state)

        # Completa el ciclo regresando al punto inicial
        cost += costo_total[current_state][visited_states[0]]
        all_routes.append(visited_states)
        all_costs.append(cost)

        # Actualización de la mejor ruta
        if cost < best_cost:
            best_cost = cost
            best_route = visited_states

    # Actualización de feromonas
    pheromones *= (1 - evaporation_rate)  # Evaporación de feromonas
    for route, cost in zip(all_routes, all_costs):
        for i in range(n_states - 1):
            pheromones[route[i]][route[i + 1]] += pheromone_constant / cost
        # Retorno al punto inicial
        pheromones[route[-1]][route[0]] += pheromone_constant / cost

# Convertir la ruta a nombres de estados y mostrar el ciclo completo
best_route_names = [nombres_estados[i] for i in best_route]
# Agregar explícitamente el retorno al estado inicial
best_route_names.append(best_route_names[0])

print("\nRuta óptima hallada con retorno al punto de partida:")
print(" → ".join(best_route_names))

print("\nDetalle del costo por tramo:")
total_cost = 0
print("\n{:<35} {:<35} {:<15}".format("Punto de origen", "Destino", "Costo (MXN)"))
print("-" * 85)

for i in range(len(best_route)):
    current_state = best_route[i]
    next_state = best_route[0] if i == len(best_route) - 1 else best_route[i + 1]
    cost_segment = costo_total[current_state][next_state]
    total_cost += cost_segment
    print("{:<35} {:<35} ${:<15,.2f}".format(
        nombres_estados[current_state],
        nombres_estados[next_state],
        cost_segment
    ))

print("\nResumen del recorrido:")
print(f"Total de estados visitados: {len(best_route_names)} (incluyendo el regreso al inicio)")
print(f"Gasto total del trayecto: ${total_cost:,.2f} MXN")
print(f"Costo medio por tramo: ${total_cost/len(best_route):,.2f} MXN")



C:\Users\Julia\AppData\Local\Temp\ipykernel_32224\3783602889.py:19: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / costo_total



Ruta óptima hallada con retorno al punto de partida:
Aguascalientes → Zacatecas → Victoria de Durango → Chihuahua → Hermosillo → Mexicali → La Paz → Culiacán Rosales → Tepic → Guadalajara → Colima → Morelia → Guanajuato → Santiago de Querétaro → Toluca de Lerdo → CDMX → Cuernavaca → Chilpancingo de los Bravo → Pachuca de Soto → Tlaxcala de Xicohténcatl → Heroica Puebla de Zaragoza → Xalapa-Enríquez → Oaxaca de Juárez → Tuxtla Gutiérrez → Villahermosa → San Francisco de Campeche → Mérida → Chetumal → Ciudad Victoria → Monterrey → Saltillo → San Luis Potosí → Aguascalientes

Detalle del costo por tramo:

Punto de origen                     Destino                             Costo (MXN)    
-------------------------------------------------------------------------------------
Aguascalientes                      Zacatecas                           $311.71         
Zacatecas                           Victoria de Durango                 $734.24         
Victoria de Durango                 C

In [21]:
import folium
from folium.plugins import AntPath

# Coordenadas aproximadas de cada estado en formato (lat, lon) - ejemplo de coordenadas en México
#crear diccionario apartir de la lista de coordenadas
coordenadas_estados = {estado: (lat, lon) for estado, lat, lon in zip(nombres_estados, coordenadas['latitude'], coordenadas['longitude'])}


# Ruta del recorrido en orden
ruta = best_route_names  # Reemplaza con tu ruta optimizada

# Inicializar el mapa centrado en México
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

inicio_coord = coordenadas_estados[ruta[0]]
folium.Marker(
    location=inicio_coord,
    popup="Inicio",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# Añadir un marcador especial para el fin (último estado en la ruta)
fin_coord = coordenadas_estados[ruta[-1]]
folium.Marker(
    location=fin_coord,
    popup="Fin",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)


for estado in ruta[1:-1]:  # Excluir inicio y fin para evitar superposición
    coord = coordenadas_estados[estado]
    folium.Marker(location=coord, popup=f"Estado {estado}").add_to(m)

# Construir la ruta animada con AntPath
ruta_coords = [coordenadas_estados[estado] for estado in ruta]
AntPath(ruta_coords, color="red", weight=4, opacity=0.8).add_to(m)
m.save('./Trayectorias_animadas_de_colonias_de_hormigas.html')
display(m)

Tanto el algoritmo de Colonia de Hormigas como el Algoritmo Genético lograron encontrar soluciones factibles para el problema del vendedor viajero en México.

**Rendimiento del Algoritmo**

- **Rendimiento del Algoritmo**
- El algoritmo genético superó al algoritmo de colonia de hormigas
- Costo final: Menor que el obtenido con colonia de hormigas ($39,639.46 MXN) a diferencia de los algoritmos genéticos ($62,588.03)